In [513]:
import numpy as np

def categorical(p):
    return np.argmax(np.random.multinomial(1, p))

class HMM:
    
    def __init__(self, pi, A, B):
        self.pi = np.array(pi)
        self.A = np.array(A)
        self.B = np.array(B)
        self.num_z = len(A)
        self.num_o = len(B[0])
        
    def sampling(self, n):
        S = [categorical(self.pi)]
        O = [categorical(self.B[S[-1]])]
        for t in range(1, n):
            S.append(categorical(self.A[S[-1]]))
            O.append(categorical(self.B[S[-1]]))
        return S, O
        
    def forward_evaluation(self, O):
        alpha = np.zeros((self.num_z, ), dtype=np.float)
        for i in range(self.num_z):
            alpha[i] = self.pi[i] * self.B[i][O[0]]
        for t in range(1, len(O)):
            cache = np.copy(alpha)
            alpha.fill(0)
            for j in range(self.num_z):
                for i in range(self.num_z):
                    alpha[j] += self.A[i][j] * cache[i]
                alpha[j] *= self.B[j][O[t]]
        return alpha.sum()
    
    def backward_evaluation(self, O):
        beta = np.ones((self.num_z, ), dtype=np.float)
        for t in reversed(range(len(O) - 1)):
            cache = np.copy(beta)
            beta.fill(0)
            for j in range(self.num_z):
                for i in range(self.num_z):
                    beta[i] += self.A[i][j] * cache[j] * self.B[j][O[t+1]]
        cache = np.copy(beta)
        beta.fill(0)
        for i in range(self.num_z):
            beta[i] = cache[i] * self.B[i][O[0]] * self.pi[i]
        return beta.sum()
    
    
    def normalized_forward_evaluation(self, O):
        alpha = np.zeros((self.num_z, ), dtype=np.float)
        normed = 1.0
        for i in range(self.num_z):
            alpha[i] = self.pi[i] * self.B[i][O[0]]
        normed = normed * alpha.sum()
        alpha = alpha / alpha.sum()
        for t in range(1, len(O)):
            cache = np.copy(alpha)
            alpha.fill(0)
            for j in range(self.num_z):
                for i in range(self.num_z):
                    alpha[j] += self.A[i][j] * cache[i]
                alpha[j] *= self.B[j][O[t]]
            normed = normed * alpha.sum()
            alpha = alpha / alpha.sum()            
        return normed
    
    def normalized_backward_evaluation(self, O):
        beta = np.ones((self.num_z, ), dtype=np.float)        
        norm = beta.sum()
        beta = beta/beta.sum()
        for t in reversed(range(len(O) - 1)):
            cache = np.copy(beta)
            beta.fill(0)
            for j in range(self.num_z):
                for i in range(self.num_z):
                    beta[i] += self.A[i][j] * cache[j] * self.B[j][O[t+1]]
            norm = norm * beta.sum()
            beta = beta/beta.sum()
        cache = np.copy(beta)
        beta.fill(0)
        for i in range(self.num_z):
            beta[i] += cache[i] * self.B[i][O[0]] * self.pi[i]
        norm = norm * beta.sum()
        return norm
    
    def viterbi_algorithm(self, O):
        delta = np.zeros((self.num_z, ), dtype=np.float)
        score = np.zeros((self.num_z, ), dtype=np.float)
        best = np.zeros((len(O), self.num_z), dtype=np.int)
        
        for i in range(self.num_z):
            delta[i] = self.pi[i] * self.B[i][O[0]]
            
        for t in range(1, len(O)):
            cache = np.copy(delta)
            delta.fill(0)
            for j in range(self.num_z):
                score.fill(0)
                for i in range(self.num_z):
                    score[i] = self.A[i][j] * cache[i] * self.B[j][O[t]]
                best[t][j] = np.argmax(score)
                delta[j] = score[best[t][j]]
                
        S = [np.argmax(delta)]
        for t in reversed(range(1, len(O))):
            S.append(best[t][S[-1]])
        S.reverse()
        return S, delta.max()
    
    def joint_probability(self, S, O):
        p = self.pi[S[0]] * self.B[S[0]][O[0]]
        for t in range(1, len(O)):
            p = p * self.A[S[t-1]][S[t]] * self.B[S[t]][O[t]]
        return p
    
#     def baum_welch(self, D):      
#         for ep in range(5):
#             NEW_A = np.zeros((self.num_z, self.num_z), dtype=np.float)
#             NEW_B = np.zeros((self.num_z, self.num_o), dtype=np.float)
#             GAMMA_1 = np.zeros((self.num_z, ), dtype=np.float)
#             GAMMA = np.zeros((self.num_z, ), dtype=np.float)
#             ETA = np.zeros((self.num_z, self.num_z), dtype=np.float)
#             GAMMA_O = np.zeros((self.num_z, ), dtype=np.float)
#             for d in range(len(D)):
#                 alpha = np.zeros((len(D[d]), self.num_z), dtype=np.float)
#                 beta = np.zeros((len(D[d]), self.num_z), dtype=np.float)
#                 gamma = np.zeros((len(D[d]), self.num_z), dtype=np.float)
#                 eta = np.zeros((len(D[d]), self.num_z, self.num_z), dtype=np.float)
#                 for i in range(self.num_z):
#                     alpha[0][i] = self.pi[i] * self.B[i][D[d][0]]
#                 for t in range(1, len(D[d])):
#                     for j in range(self.num_z):
#                         for i in range(self.num_z):
#                             alpha[t][j] += self.A[i][j] * alpha[t-1][i]
#                         alpha[t][j] *= self.B[j][D[d][t]]

#                 for i in range(self.num_z):
#                     beta[len(D[d]) - 1][i] = 1.0

#                 for t in reversed(range(len(D[d]) - 1)):
#                     for j in range(self.num_z):
#                         for i in range(self.num_z):
#                             beta[t][i] += self.A[i][j] * beta[t+1][j] * self.B[j][D[d][t+1]]
                            
#                 for i in range(self.num_z):
#                     beta[0][i] = beta[1][i] * self.B[i][D[d][0]] * self.pi[i]

#                 for t in range(len(D[d])):
#                     denom = 0.0
#                     for i in range(self.num_z):
#                         gamma[t][i] = alpha[t][i] * beta[t][i]
#                         denom += gamma[t][i]
#                     gamma[t] /= denom

#                 for t in range(len(D[d]) - 1):
#                     denom = 0.0
#                     for i in range(self.num_z):
#                         for j in range(self.num_z):
#                             eta[t][i][j] = alpha[t][i] * self.A[i][j] * self.B[j][D[d][t+1]] * beta[t + 1][j]
#                             denom += eta[t][i][j]
#                     eta[t] /= denom

#                 for i in range(self.num_z):
#                     GAMMA_1[i] += gamma[0][i]

#                 for t in range(len(D[d])):
#                     for i in range(self.num_z):
#                         GAMMA[i] += gamma[t][i]
#                         NEW_B[i][D[d][t]] += gamma[t][i]
#                         for j in range(self.num_z):
#                             NEW_A[i][j] += eta[t][i][j]

#             GAMMA_1 = GAMMA_1 / GAMMA_1.sum()
#             for k in range(self.num_z):
#                 NEW_B[k] = NEW_B[k] / NEW_B[k].sum()
#                 NEW_A[k] = NEW_A[k] / NEW_A[k].sum()
        
#             self.pi = GAMMA_1
#             self.A = NEW_A
#             self.B = NEW_B

            
            
            
            

In [514]:
pi = [0.6, 0.4]

A = [[0.7, 0.3],
     [0.4, 0.6]]

B = [[0.1, 0.4, 0.5],
     [0.6, 0.3, 0.1]]

hmm = HMM(pi, A, B)

# D = []
# for d in range(10):
#     _, O = hmm.sampling(200)
#     D.append(O)


In [515]:
S_ = ['비', '해']
O_ = ['산책', '쇼핑', '연구'] 

S, O = hmm.sampling(200)
V, _ = hmm.viterbi_algorithm(O)
print(_)
print(hmm.joint_probability(S, O))
print(hmm.joint_probability(V, O))

for i in range(50):
    print(S_[S[i]], O_[O[i]], S_[V[i]])


4.8904135402034755e-117
2.001512341932144e-134
4.8904135402034755e-117
비 연구 비
비 연구 비
비 산책 해
해 산책 해
비 쇼핑 비
비 연구 비
해 산책 해
해 쇼핑 비
비 연구 비
비 연구 비
비 쇼핑 비
비 연구 비
비 쇼핑 비
비 산책 해
비 연구 비
비 연구 비
해 산책 해
비 연구 비
비 산책 해
비 연구 비
비 연구 비
해 쇼핑 비
해 산책 해
해 산책 해
비 쇼핑 비
비 쇼핑 비
해 쇼핑 비
해 연구 비
해 산책 해
비 쇼핑 해
비 산책 해
비 쇼핑 비
해 쇼핑 비
비 연구 비
비 연구 비
비 쇼핑 비
해 산책 해
해 쇼핑 비
비 연구 비
해 연구 비
해 산책 해
해 산책 해
해 연구 비
해 산책 해
비 쇼핑 비
비 쇼핑 비
비 연구 비
비 연구 비
비 연구 비
해 산책 해


In [516]:
print(hmm.forward_evaluation(O))
print(hmm.backward_evaluation(O))
print(hmm.normalized_forward_evaluation(O))
print(hmm.normalized_backward_evaluation(O))

1.2596241667977616e-95
1.25962416679776e-95
1.259624166797763e-95
1.259624166797764e-95
